# NLP Launch: Natural Language Predictor Project

## Goal: 
* The project goal is to predict which programming language is used based solely on GitHub README's. 
* This analysis aims to address the listed questions and uncover any additional important findings related goal.

## Imports

In [1]:
#.py modules
import acquire as ac
# import explore as ex

#numbers
import pandas as pd
import numpy as np

#vizzes
import matplotlib.pyplot as plt
import seaborn as sns

Imports Successful


> # `Acquire`

### Acquire
* Data acquired from GitHub using webscraping techniques
* Shape before cleaning
* Shape after cleaning


### Data Dictionary

| Feature | Definition|
|:--------|:-----------|
|repo| The 'author name/repo name' of the repo|
|language| The repo language used|
|readme_contents| The contents of the entire README from each site|
|clean| The cleaned version of readme_contents|
|lemma| The lemmatized version of the clean column|

In [18]:
# #acquiring


In [1]:
# #quick peek
# df.head()

In [2]:
# #descriptive statistics
# df.describe()

In [ ]:
# #df shape
# df.shape

> # `Prepare`

### Prepare
* Prepare
    * dropped columns
        * insert
    * renamed columns
        * insert
        * insert
        * insert
    * changed dates to datetime type
    * created new columns
        * insert
        * insert
        * insert
    * no nulls

In [ ]:
# #prepare


In [ ]:
# #clean


In [ ]:
# #quick peek


In [20]:
# #univariate exploration?


In [21]:
# #distributions


> # `Explore`

### Questions To Answer:
1. What are the most common words in READMEs?
2. Does the length of the README vary by programming language?
3. Do different programming languages use a different number of unique words?
4. Are there any words that uniquely identify a programming language?

## 1. What are the most common words in READMEs?

In [3]:
# #visualize the most used words


In [16]:
# #hypothesize


In [ ]:
# #analyze?


### Insight:

<br>

<br>


## 2. Does the length of the README vary by programming language?

In [4]:
# #visualize the lengths of readmes against labels


In [ ]:
# #hypothesize


In [15]:
# #analyze?


### Insight:

<br>

<br>

## 3. Do different programming languages use a different number of unique words?

In [22]:
# #visualize number of unique words per label


In [10]:
# #hypothesize


In [11]:
# #analyze?


### Insight:

<br>

<br>

## 4. Are there any words that uniquely identify a programming language?

In [23]:
# #visualize top words per label


In [13]:
# #hypothesize


In [17]:
# #analyze?


### Insight:

<br>

<br>

> # `Conclusion`

## Explore

* 
* 
* 
* 
* 
* 

> # `Recommendations/Next Steps`

* 
* 
* 